## PYHTON DB API

### PYODBC

In [ ]:
pip install pyodbc

In [1]:
import pyodbc

## SQL Server Authentication

In [ ]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [ ]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost'
database = 'master'
user = 'sa'
password = '----'
conn = create_server_connection(driver, server, database, user, password)


In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'Amazon Redshift (x64)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

## WINDOWS AUTHENTICATION

In [2]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [3]:
conn = pyodbc.connect(conn_string);

## cursor process

In [4]:
crs = conn.cursor()

In [5]:
conn.autocommit = True

In [6]:
conn.commit()

### create database

In [7]:
def create_database(conn, create_database_query): #parametreler: connection nesnesi ve query nesnesi
    crs = conn.cursor() #bir cursor nesnesi oluşturuyoruz. bunu query sonucundaki değerleri yakalamak için kullanacağız.
    try:
        crs.execute(create_database_query)#query'yi çalıştır sonucu yakala. queryleri excecute ile çalıştırıyoruz
        print("Database is Created Successfuly")
    except Error as err:
            print(f"Error:'{err}'")

In [8]:
create_database_query = 'CREATE DATABASE Testc'

In [9]:
create_database(conn, create_database_query)

Database is Created Successfuly


In [10]:
query = 'USE Testc'

In [11]:
crs.execute(query)

In [12]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [13]:
execute_query(conn, query)

Query Succeessful!


In [14]:
query = 'CREATE TABLE TestTable (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

In [15]:
execute_query(conn, query)

Query Succeessful!


## Insert

In [16]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [17]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Bruce', 'Lee')

In [18]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## Cursor Function

In [19]:
crs.execute("DELETE FROM TestTable WHERE FirstName = 'Jerry'")

In [20]:
crs.rowcount

1

#### Fetchone

In [21]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
row = crs.fetchone() # tablodaki ilk kaydi getirir
row

('Bob', 'Marley')

In [22]:
row = crs.fetchone() # tablodaki ilk kaydi getirir
row

('Bruce', 'Lee')

### Fetchmany

In [23]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
rows = crs.fetchmany(3)
rows

[('Bob', 'Marley'), ('Bruce', 'Lee'), ('Tom', 'Cat')]

### FetchAll

In [24]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
rows = crs.fetchall()
rows

[('Bob', 'Marley'),
 ('Bruce', 'Lee'),
 ('Tom', 'Cat'),
 ('Owen', 'William'),
 ('Stefan', 'Müller')]

### Create DataFrame

In [25]:
import pandas as pd

In [26]:
result2 = crs.execute('SELECT * FROM TestTable')
result2

In [27]:
crs.execute('SELECT * FROM TestTable')
result = crs.fetchall()
result

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Cat'),
 (5, 'Owen', 'William'),
 (6, 'Stefan', 'Müller')]

In [28]:
df = pd.DataFrame(result)

In [29]:
df

,0
0,"[1, Bob, Marley]"
1,"[2, Bruce, Lee]"
2,"[3, Tom, Cat]"
3,"[5, Owen, William]"
4,"[6, Stefan, Müller]"


In [30]:
a = []

for result in result:
    result = list(result)
    a.append(result)

In [31]:
a

[[1, 'Bob', 'Marley'],
 [2, 'Bruce', 'Lee'],
 [3, 'Tom', 'Cat'],
 [5, 'Owen', 'William'],
 [6, 'Stefan', 'Müller']]

In [32]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(a, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,3,Tom,Cat
3,5,Owen,William
4,6,Stefan,Müller


In [33]:
query = 'USE SampleRetail'
execute_query(conn, query)

Query Succeessful!


In [34]:
df_prod = pd.read_sql("SELECT * FROM product.product", con =conn)
df_prod.head(8)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 ...,23,4,2021,23.99
1,2,NS-SP1800BL 5.1-Channel Home Theater System (B...,5,5,2021,136.99
2,3,Acoustimass 6 Series V Home Theater Speaker Sy...,24,5,2021,599.00
3,4,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,6,4,2021,151.99
4,5,Details About Samsung Gear Iconx 2018 Edition ...,1,14,2021,199.99
5,6,2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,12,4,2021,89.95
6,7,Details About Alpine 480w 6.5 2way Typee Coaxi...,35,5,2021,59.99
7,8,Kenwood KFC-1653MRW 6.5 2-way Marine Speakers ...,31,5,2021,99.99
